In [1]:
#!/usr/bin/env python3
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    
    #Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    #Load session
    tf.saved_model.loader.load(sess,[vgg_tag],vgg_path)
    
    #Get the graph
    graph = tf.get_default_graph()
    
    #Create the session tensors
    image_input = sess.graph.get_tensor_by_name(vgg_input_tensor_name)
    keep= sess.graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    final_vgg_layer3_out = sess.graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    final_vgg_layer4_out = sess.graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    final_vgg_layer7_out = sess.graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    #Return them
    return image_input, keep, final_vgg_layer3_out, final_vgg_layer4_out, final_vgg_layer7_out

tests.test_load_vgg(load_vgg, tf)


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    
    #Perform the 1x1 convolutions
    
    layer7_1x1=tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1, kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

    layer4_1x1=tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1, kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

    layer3_1x1=tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1, kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #First transpose convolution --- Kernel size = 4 / strides = (2,2)
    
    output1 = tf.layers.conv2d_transpose(layer7_1x1, num_classes, 4,(2,2), 'SAME', kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #First skip layer
    
    skip_layer_4 = tf.add(output1, layer4_1x1)
    
    #Second transpose convolution --- Kernel size = 4 / strides = (2,2)
    
    output2 = tf.layers.conv2d_transpose(skip_layer_4, num_classes, 4,(2,2), 'SAME', kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

    #Second skip layer

    skip_layer_3 = tf.add(output2, layer3_1x1)
    
    #Produce final --- Kernel size = 16 / strides = (8,8)

    final_output = tf.layers.conv2d_transpose(skip_layer_3, num_classes, 16,(8,8), 'SAME', kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    return final_output

tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    
    # Reshape logits and labels
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    
    #Use cross entropy
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
    
    #Use Adam Optimizer for optimizing and minimine losses
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss

tests.test_optimize(optimize)


def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    
    #Training the model
    for epoch in range(epochs):
        print("Epoch: {}".format(epoch+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict={
                               input_image: image, correct_label: label, keep_prob: 0.6, learning_rate: 1e-4})
            print('Training loss: {:.3f}'.format(loss))

tests.test_train_nn(train_nn)


def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    
    #Set the tunable parameters
    epochs = 30
    batch_size = 6

    with tf.Session() as sess:
        
        #Set the labels, keep_prob and learning rate
        correct_label = tf.placeholder(dtype=tf.float32, shape=(None, None, None, num_classes))
        learning_rate = tf.placeholder(dtype=tf.float32)
        keep_prob = tf.placeholder(tf.float32)
        
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out =  load_vgg(sess, vgg_path);
        
        # Build Fully Convolutional Network
        output_final = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)
        
        #Call optimize
        logits, train_op, cross_entropy_loss = optimize(output_final, correct_label, learning_rate, num_classes)
        
        #Train NN using the train_nn function
        sess.run(tf.global_variables_initializer())
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate)

        #Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)



if __name__ == '__main__':
    run()


TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0
Tests Passed
Tests Passed
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
